# Week 1 - #01 Text Preprocessing

**Part I:** [Text Preprocessing with NLTK](https://www.nltk.org/book/ch03.html)

- Tokenizer
- Vectorization
- Stopwords
- Lemmatization
- Normalization


**Part II:** [Scikit-learn: Text Feature Extraction](https://scikit-learn.org/stable/modules/feature_extraction.html)

- Bagofwords
- TFIDF
- Unigram
- Bigram

## Assignment:

Dataset: [Vietnamese Online News Dataset](https://www.kaggle.com/datasets/haitranquangofficial/vietnamese-online-news-dataset)

- **Part I**: Preprocess a given text dataset and report the steps and results.
- **Part II**: Apply 4 techniques on a given dataset

## Import dataset

In [1]:
import json
DAT_PATH = 'news_dataset.json'
with open(DAT_PATH, 'r') as f:
    dat_raw = json.load(f)

In [2]:
print(f"Number of items in json file: {len(dat_raw)}")

Number of items in json file: 184539


In [3]:
print(f"Metadata: {dat_raw[0].keys()}")

Metadata: dict_keys(['id', 'author', 'content', 'picture_count', 'processed', 'source', 'title', 'topic', 'url', 'crawled_at'])


In [4]:
dat_raw[0]

{'id': 218270,
 'author': '',
 'content': "Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường

In [5]:
# check missing content
empty_content_num = sum(1 for item in dat_raw if not item.get("content", "").strip())
print(f"Number of new documents that content is empty: {empty_content_num}")

Number of new documents that content is empty: 23468


In [6]:
# remove empty content items
raw_news = [doc for doc in dat_raw if doc.get("content", "").strip()]
len(raw_news)

161071

## Text Preprocessing

### Part I: Text Preprocessing

Pipeline được đóng gói thành 2 class chính:
- `TextProcessor`: xử lý đơn lẻ một văn bản.
- `NewsProcessor`: xử lý toàn bộ tập dữ liệu tin tức (list các document).

**`TextProcessor`**:

- Normalize:
  - Lowercase
  - Xóa email, URL, domain.
  - Xóa thông tin không liên quan (giấy phép, chính sách...).
  - Xóa số, ký tự đặc biệt, chuẩn hóa khoảng trắng.
- Tokenize: tách từ bằng underthesea.
- Remove stopwords: loại bỏ từ dừng.

**`NewsProcessor`**:

- Process:
  - Xử lý toàn bộ document, lưu {id, content}.
  - In log cho 30 doc đầu + sau mỗi 1000 doc.
  - Hỗ trợ early_stopping.

- Save:
  - vectorizer_flag=True, lưu CSV (id, content).
  - vectorizer_flag=False, lưu JSON (list dict).

In [7]:
# pip install underthesea

In [9]:
# Load class TextProcessor and NewsProcessor from file text_processor.py
from text_processor import TextProcessor, NewsProcessor

#### I.1 Test TextProcessor class

In [10]:
text_processor = TextProcessor(vectorizer_flag=False)

sample = raw_news[0]['content']
processed_sample = text_processor.process(sample)

print(f"Original Content: \n{sample}\n")
print(f"Processed Content: \n{processed_sample}\n")

Original Content: 
Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người đua nhau n

#### I.2 Applied preprocessing text for 1000 first documents in dataset

In [11]:
news_processor = NewsProcessor(stopwords_path="vietnamese-stopwords.txt", vectorizer_flag=False)

processed_news = news_processor.process(raw_news, early_stopping=1000)
news_processor.save(processed_news, "processed_tokens.json")

Document 1:
Original content: Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người

In [12]:
news_processor = NewsProcessor(stopwords_path="vietnamese-stopwords.txt", vectorizer_flag=True)

processed_news = news_processor.process(raw_news, early_stopping=1000)
news_processor.save(processed_news, "processed_news.csv")

Document 1:
Original content: Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã có thông tin ban đầu về vụ nổ súng,cướp tiệm vàng tại chợ Đông Ba nằm trên đường Trần Hưng Đạo (TP Huế, tỉnh Thừa Thiên - Huế). Thông Sài Gòn Giải Phóng, khoảng 12h30' ngày 31/7, một đối tượng sử dụng súng AK bất ngờ xông vào tiệm vàng Hoàng Đức và Thái Lợi (phía trước chợ Đông Ba) rồi nổ súng chỉ thiên liên tiếp uy hiếp chủ tiệm để cướp vàng. Sau đó, đối tượng mang số vàng vừa cướp được vứt ra vỉa hè rồi đi bộ đến khu vực cầu Gia Hội, cách khu vực gây án khoảng 300m. Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức trực tiếp chỉ đạo các lực lượng chức năng gồm Công an tỉnh và Công an TP Huế nhanh chóng có mặt tại hiện trường triển khai đồng bộ các biện pháp nghiệp vụ, khoanh vùng và ngăn không để người dân đi vào hiện trường. Hàng trăm tiểu thương trong chợ Đông Ba và người dân gần cầu Gia Hội được yêu cầu di chuyển khỏi hiện trường, đóng cửa nhà đề phòng đạn lạc. Tuy nhiên, thấy vàng bị ném ra đường, nhiều người

### Part II: Text Feature Extraction

#### Read cleaned dataset file saved from above

In [13]:
import pandas as pd
df_news = pd.read_csv('processed_news.csv')
df_news.head()

,id,content
0,218270,chiều công an tỉnh thừa thiên huế thông tin ba...
1,218269,thứ trưởng phát triển kỹ thuật số truyền thông...
2,218268,kết quả thi tốt nghiệp thpt trung bình môn toá...
3,218267,thống đốc kentucky andy beshear hôm đợt mưa lũ...
4,218266,vụ tai nạn giao thông liên hoàn phố đi tam bạc...


In [17]:
texts = df_news['content'].astype(str).tolist()

#### Apply 4 Text Feature Extraction Techniques

- BagofWords (BoW)
- TF-IDF
- Unigram
- Bigram

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

##### Bag of Words (BoW)

In [18]:
bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(texts)

In [61]:
# 1. Vocab and features matrix size
print("The number of words in the entire dataset:", len(bow_vectorizer.vocabulary_))
print("Feature matrix size:", X_bow.shape)

# 2. Sparse and Dense
density = X_bow.nnz / (X_bow.shape[0] * X_bow.shape[1])
print(f"Matrix density: {density:.6f}")

# 3. Top features
sum_words = X_bow.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in bow_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
print("Top 10 most common features:\n", pd.DataFrame(words_freq[:10], columns=['Word', 'Fequency']))

The number of words in the entire dataset: 8205
Feature matrix size: (1000, 8205)
Matrix density: 0.019655
Top 10 most common features:
      Word  Fequency
0    công      2943
1    đồng      1566
2     nam      1551
3   thành      1443
4     gia      1432
5  trường      1393
6     đầu      1365
7    động      1350
8    hàng      1282
9     học      1266


#### TF-IDF

In [72]:
tfidf_vectorizer = TfidfVectorizer(smooth_idf=False)
X_tfidf = tfidf_vectorizer.fit_transform(texts)

In [73]:
# 1. Vocab and features matrix size
print("The number of words in the entire dataset:", len(tfidf_vectorizer.vocabulary_))
print("Feature matrix size:", X_tfidf.shape)

# 2. Sparse and Dense
density = X_tfidf.nnz / (X_tfidf.shape[0] * X_tfidf.shape[1])
print(f"Matrix density: {density:.6f}")

The number of words in the entire dataset: 8205
Feature matrix size: (1000, 8205)
Matrix density: 0.019655


In [74]:
# 3. Top features per doc
feature_names = tfidf_vectorizer.get_feature_names_out()
top_words = []
for i in range(X_tfidf.shape[0]):
    row = X_tfidf[i].toarray().flatten()
    top_idx = row.argmax()  # index của từ có TF-IDF lớn nhất
    top_word = feature_names[top_idx]
    top_value = row[top_idx]
    print(f"Doc {i+1}: Most distinctive word is '{top_word}' - TF-IDF = {top_value:.4f}")
    top_words.append((i, top_word, top_value))

Doc 1: Most distinctive word is 'huế' - TF-IDF = 0.3796
Doc 2: Most distinctive word is 'nga' - TF-IDF = 0.3892
Doc 3: Most distinctive word is 'môn' - TF-IDF = 0.3547
Doc 4: Most distinctive word is 'kentucky' - TF-IDF = 0.5800
Doc 5: Most distinctive word is 'xe' - TF-IDF = 0.5272
Doc 6: Most distinctive word is 'đức' - TF-IDF = 0.4469
Doc 7: Most distinctive word is 'hân' - TF-IDF = 0.3315
Doc 8: Most distinctive word is 'liverpool' - TF-IDF = 0.4756
Doc 9: Most distinctive word is 'ronaldo' - TF-IDF = 0.3967
Doc 10: Most distinctive word is 'nỗi' - TF-IDF = 0.3847
Doc 11: Most distinctive word is 'hội' - TF-IDF = 0.4849
Doc 12: Most distinctive word is 'ukraine' - TF-IDF = 0.4767
Doc 13: Most distinctive word is 'tùng' - TF-IDF = 0.3790
Doc 14: Most distinctive word is 'bđs' - TF-IDF = 0.2633
Doc 15: Most distinctive word is 'iphone' - TF-IDF = 0.6563
Doc 16: Most distinctive word is 'giọng' - TF-IDF = 0.5013
Doc 17: Most distinctive word is 'cà' - TF-IDF = 0.3045
Doc 18: Most dist

##### Unigram

In [37]:
# Unigram model using CountVectorizer
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))  # Only unigrams
X_unigram = unigram_vectorizer.fit_transform(texts)

In [68]:
# 1. Vocab and features matrix size
print("The number of words in the entire dataset:", len(unigram_vectorizer.vocabulary_))
print("Feature matrix size:", X_unigram.shape)

# 2. Sparse and Dense
density = X_unigram.nnz / (X_unigram.shape[0] * X_unigram.shape[1])
print(f"Matrix density: {density:.6f}")

# 3. Top features
sum_words = X_unigram.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in unigram_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
print("Top 10 most common features:\n", pd.DataFrame(words_freq[:10], columns=['Word', 'Fequency']))

The number of words in the entire dataset: 8205
Feature matrix size: (1000, 8205)
Matrix density: 0.019655
Top 10 most common features:
      Word  Fequency
0    công      2943
1    đồng      1566
2     nam      1551
3   thành      1443
4     gia      1432
5  trường      1393
6     đầu      1365
7    động      1350
8    hàng      1282
9     học      1266


In [66]:
pd.DataFrame(X_unigram.toarray(), columns = unigram_vectorizer.get_feature_names_out())

,aa,aachen,aarhus,aaroeen,aaroen,ab,ababil,abc,abdala,abdul,...,ộp,ớn,ớt,ủi,ủng,ủy,ức,ứng,ứớc,ửng
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Bigram

In [40]:
# Bigram model using CountVectorizer
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))  # Only bigrams
X_bigram = bigram_vectorizer.fit_transform(texts)

In [69]:
# 1. Vocab and features matrix size
print("The number of words in the entire dataset:", len(bigram_vectorizer.vocabulary_))
print("Feature matrix size:", X_bigram.shape)

# 2. Sparse and Dense
density = X_bigram.nnz / (X_bigram.shape[0] * X_bigram.shape[1])
print(f"Matrix density: {density:.6f}")

# 3. Top features
sum_words = X_bigram.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in bigram_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
print("Top 10 most common features:\n", pd.DataFrame(words_freq[:10], columns=['Word', 'Fequency']))

The number of words in the entire dataset: 127412
Feature matrix size: (1000, 127412)
Matrix density: 0.001958
Top 10 most common features:
         Word  Fequency
0   việt nam       859
1     hà nội       576
2    công an       517
3    công ty       492
4    khu vực       448
5  thông tin       439
6  người dân       406
7    tổ chức       396
8    cơ quan       374
9   gia đình       354


In [67]:
pd.DataFrame(X_bigram.toarray(), columns = bigram_vectorizer.get_feature_names_out())

,aa adder,aa alamo,aa aphid,aa archer,aachen học,aarhus đan,aaroeen ronaldo,aaroen chính,ab portland,ababil iii,...,ứng đăng,ứng đầu,ứng đầy,ứng đẩy,ứng đối,ứng đồng,ứng đội,ứng đừng,ứớc tính,ửng đỏ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
